In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.path import Path
from multiprocess import Pool

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from data_manager import *
from metadata import *
from cell_utilities import *
from learning_utilities import *

/oasis/projects/nsf/csd395/yuncong/brain_virtualenv/lib/python2.7/site-packages/matplotlib/__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Setting environment for Gordon


No vtk
Cannot import mxnet.


In [2]:
patch_size = 224
half_size = patch_size/2
patch_spacing = 56

In [3]:
# for stack in all_annotated_stacks:
for stack in ['MD594']:
    
    grid_spec = (patch_size, patch_spacing, ) + metadata_cache['image_shape'][stack]
    grid_locations = grid_parameters_to_sample_locations(grid_spec)

    first_section, last_section = metadata_cache['section_limits'][stack]
    
#     for sec in range(first_section, last_section+1):
    for sec in range(204, 205):

        if is_invalid(sec=sec, stack=stack):
            continue
            
        mask_tb = DataManager.load_thumbnail_mask_v2(stack, sec)
        region_patch_indices = locate_patches_v2(stack=stack, grid_spec=grid_spec, mask_tb=mask_tb)
        
        # Patch as regions
        region_patch_locations = grid_locations[region_patch_indices]
        ul = region_patch_locations + (-half_size, -half_size)
        ur = region_patch_locations + (-half_size, half_size)
        lr = region_patch_locations + (half_size, half_size)
        ll = region_patch_locations + (half_size, -half_size)
        
        region_contours = np.array(zip(ul, ur, lr, ll))
        
        t = time.time()
        compute_features_regions(stack, sec, region_contours)
        sys.stderr.write('Time: %d seconds\n' % (time.time()-t)) # 257 seconds
        
        locations_fp = get_cell_classifier_data_filepath('region_locations', stack=stack, sec=sec, ext='txt')
#         np.savetxt(locations_fp, np.c_[region_patch_indices, region_patch_locations], fmt='%d %d %d')

In [6]:
region_contours.shape

(43616, 4, 2)